In [33]:
from pyampd.ampd import find_peaks #Normal AMPD algorithm
import matplotlib.pyplot as plt    #Plotting capabilities
import numpy as np                 #Mathematical operations on large arrays
from numpy import sin, pi
import statistics                  #allows for statistical operations

#Let's keep all odd numbered sensor bands to be cerebral and even numbered bands to be abdominal
class One:
    signals = [[],[],[]]
    filt_signals = [[],[],[]]
    time = []
    R_peaks = []
    StO2 = []
    bpm = []
    hrv = []
    temp = []
    cs_ratio = []
    pair = ""
    stored_nums = [-1,-1] #List order: peaks, cs_ratio (correspond to the numbers used for each function)
class Two:
    signals = [[],[],[]]
    filt_signals = [[],[],[]]
    time = []
    R_peaks = []
    StO2 = []
    bpm = []
    hrv = []
    temp = []
    cs_ratio = []
    pair = ""
    stored_nums = [-1,-1]
    
myDict = {"1": One, "2": Two}

def fake_timeseries(tmax=30, seed=None):
    #Generate fake data for testing peak detection algorithms
    np.random.seed(seed)
    fs, f1, f2, f3, a, b, c, d = 30, 1, 2, 0.1, 1, 0.6, 1, 1.
    N = int(tmax*fs)
    t = np.linspace(0, tmax, N)
    x = a*sin(2*pi*f1*t) + b*sin(2*pi*f2*t) + c*sin(2*pi*f3*t) + d*np.random.rand(N) + 20
    return t, x

#Runs continuously
def avg_filter(band, num_measurements):
    for i in range(0,3):
        total = 0
        for j in range(0,num_measurements): #Adds the first few numbers in the list, then removes them
            total += band.signals[i].pop(0)
            
        band.filt_signals[i].append(total/num_measurements)

#Runs continuously
def StO2(band):
    '''
    Refer to "1/11 StO2 Equation" from Fall 2023 for the math.
    Reference: https://patentimages.storage.googleapis.com/1f/85/02/e884e4fec6d945/US6456862.pdf
    
    Absorbance coefficients for oxyhemoglobin and deoxyhemoglobin at 700 nm, 850 nm, & 890 nm, respectively.
    Reference: https://www.sciencedirect.com/science/article/pii/0005272888900692?via%3Dihub
    '''
    
    coeff_HbO2 = (0.421, 1.097, 1.226) #Tuples require less memory and are immutable
    coeff_Hb = (1.798, 0.781, 0.866)
    G = 100.03 #Arbitrary number for testing
    each_average_StO2 = []
    StO2_list = [[],[],[]]
    for i in range(0,3):
        if i == 2:
            for j in range(0,len(band.filt_signals[0])):
                AG_epsilon = (band.filt_signals[0][j] - G) * (coeff_Hb[2]*coeff_HbO2[0] - coeff_HbO2[2]*coeff_Hb[0])
                AAG_epsilons = (band.filt_signals[2][j]*coeff_HbO2[0] - band.filt_signals[0][j]*coeff_HbO2[2] + G*(coeff_HbO2[2]-coeff_HbO2[0]))
                StO2_list[2].append((AG_epsilon - coeff_Hb[0]*AAG_epsilons)/(AG_epsilon - (coeff_Hb[0]+coeff_HbO2[0])*AG_epsilon))
        else:
            for j in range(0,len(band.filt_signals[0])):
                AG_epsilon = (band.filt_signals[i][j] - G) * (coeff_Hb[i+1]*coeff_HbO2[i] - coeff_HbO2[i+1]*coeff_Hb[i])
                AAG_epsilons = (band.filt_signals[i+1][j]*coeff_HbO2[i] - band.filt_signals[i][j]*coeff_HbO2[i+1] + G*(coeff_HbO2[i+1]-coeff_HbO2[i]))
                StO2_list[i].append((AG_epsilon - coeff_Hb[i]*AAG_epsilons)/(AG_epsilon - (coeff_Hb[i]+coeff_HbO2[i])*AG_epsilon))
        
    for k in range(0,len(StO2_list[0])):
        each_average_StO2.append((StO2_list[0][k] + StO2_list[1][k] + StO2_list[2][k])/3)
    
    band.StO2.append(statistics.mean(each_average_StO2))
        

#Sleep 5 secs
def peaks(band):
    #Returns the indices of occurance only in the new section
    signal_section = band.filt_signals[0][(band.stored_nums[0]+1):]
    new_peaks = list(find_peaks(signal_section))
    band.R_peaks = new_peaks
    band.stored_nums[0] += len(signal_section)
        

#Sleep 5 secs
def bpm(band): #pass in list of connected bands, pass in desired length of time, default 5 secs
    beats = 0
    band.bpm.append(len(band.R_peaks)/(1/30))
    

#Sleep 5 secs
def cs_ratio(band):
    if band == One:
        StO2_head = band.StO2[(band.stored_nums[1]+1)]
        StO2_stomach = myDict[band.pair].StO2[(band.stored_nums[1]+1)]
        band.cs_ratio.append(StO2_head/StO2_stomach)
        myDict[band.pair].cs_ratio.append(StO2_head/StO2_stomach)
        band.stored_nums[1] += 1

    
#Sleep 5 secs
def hrv(band):
    hrv_untranslated = statistics.mean(np.diff(band.R_peaks)) #np.diff determines the difference between each peak
    hrv_translated = hrv_untranslated/0.03 #Need to translate indices to time. Assuming 30 readings (indices) per sec
    band.hrv.append(hrv_translated)

#Creating signal for band One
t0, x0 = fake_timeseries() #calling the function to make the fake signals
t1, x1 = fake_timeseries()
t2, x2 = fake_timeseries()
One.time = list(t0)

#Creating signal for band Two
t3, x3 = fake_timeseries()
t4, x4 = fake_timeseries()
t5, x5 = fake_timeseries()
Two.time = list(t3)

#User input and initialization
band_nums = []
user_input = input("Enter the numbers of the connected bands. Separate by spaces or commas: ")
my_string = user_input.replace(',',' ')
band_nums = [myDict[band_num] for band_num in my_string.split()] #myDict[band] translates the user input to the class names

user_input = input("Enter the numbers of the sensor bands that are on the same rat. Enter cerebral sensor band first. Separate with commas or spaces: ")
my_string = user_input.replace(',',' ')
pair_nums = my_string.split()
#Storing the pairs to call in the function and write to csv file later
myDict[pair_nums[0]].pair = pair_nums[1]
myDict[pair_nums[1]].pair = pair_nums[0]

#Data collection
length = 60
interval = 60
section = 2
while length <= len(t0):
    One.signals[0] += list(x0)[length-interval:length]
    One.signals[1] += list(x1)[length-interval:length]
    One.signals[2] += list(x2)[length-interval:length]
    Two.signals[0] += list(x3)[length-interval:length]
    Two.signals[1] += list(x4)[length-interval:length]
    Two.signals[2] += list(x5)[length-interval:length]
        
    for band in band_nums:
        for i in range(0,int(interval/section)):
            avg_filter(band, section)
            
        if length > 100:
            cs_ratio(band) #Doesn't run for first round of data collection since StO2 for the paired band doensn't exist yet
        
        StO2(band)
        peaks(band)
        bpm(band)
        hrv(band)
    
    length += interval

#Displaying Results
for band in band_nums:
    print("")
    print("StO2: ", band.StO2)
    print("BPM: ", band.bpm)
    print("HRV: ", band.hrv)
    print("CS Ratio: ", band.cs_ratio)
    print("Pair band: ", band.pair)

Enter the numbers of the connected bands. Separate by spaces or commas: 1 2
Enter the numbers of the sensor bands that are on the same rat. Enter cerebral sensor band first. Separate with commas or spaces: 1 2

StO2:  [4.537286365371328, 4.582869339922578, 4.577444318519695, 4.5834743383697445, 4.585982614866943, 4.582484572877769, 4.576951817941782, 4.580564769199635, 4.582015346985917, 4.582263518984918, 4.582906739615645, 4.58235575539112, 4.584650840445811, 4.583419537547499, 4.584801358364501]
BPM:  [150.0, 60.0, 150.0, 150.0, 60.0, 60.0, 60.0, 60.0, 150.0, 180.0, 150.0, 180.0, 150.0, 60.0, 180.0]
HRV:  [200.0, 466.6666666666667, 233.33333333333334, 200.0, 566.6666666666667, 566.6666666666667, 500.0, 466.6666666666667, 200.0, 166.66666666666669, 200.0, 166.66666666666669, 200.0, 433.33333333333337, 166.66666666666669]
CS Ratio:  [0.976057265451027, 0.9943799099496121, 0.9959099308930591, 0.9979815216148784, 1.000600771736034, 0.9989517719902687, 0.9968407835491196, 0.9980064454043